In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from mll_calc.mll_pred import format_XY, ratios, mll_testset

In [2]:
unc = 0.05
lbls = ['ReactorType', 'CoolingTime', 'Enrichment', 'Burnup', 'OrigenReactor']
tamu_list = ['cs137/cs133', 'cs134/cs137', 'cs135/cs137', 'ba136/ba138',
             'sm150/sm149', 'sm152/sm149', 'eu154/eu153', 'pu240/pu239',
             'pu241/pu239', 'pu242/pu239']

In [3]:
train_db = '~/sims_n_results/sfcompo_compare_aug2020/van2ohi2_not-scaled_15nuc_5.pkl'
test_db = '~/sfcompo/format_clean/sfcompo_formatted.pkl'
XY = format_XY(train_db)
test = pd.read_pickle(test_db)

ratio_list = tamu_list
XY = ratios(XY, ratio_list, lbls)
test = ratios(test, ratio_list, lbls)

## Test sample from SFCOMPO that was simulated in ORIGEN

In [4]:
test = test.loc[test.index == 'VAN-2|EF05|WZR0058|E58-88']
test

,cs137/cs133,cs134/cs137,cs135/cs137,ba136/ba138,sm150/sm149,sm152/sm149,eu154/eu153,pu240/pu239,pu241/pu239,pu242/pu239,ReactorType,CoolingTime,Enrichment,Burnup,OrigenReactor
VAN-2|EF05|WZR0058|E58-88,1.021429,0.012867,0.548252,0.0,97.640118,36.873156,0.124812,0.390879,0.184039,0.090554,pwr,0.0,4.5,43520.0,Vandellos-2_PWR


## 24 Sims from ORIGEN 

#### Only variations are: 
1. 12 cooling times, since that is unknown in SFCOMPO (ranging 1 min to 12 years)
2. 2 enrichment xsec libs (4 and 5%) in ORIGEN, just in case one worked better in interpolation scheme

#### VAN-2|EF05|WZR0058|E58-88 Simulation Parameters:
* PWR: Vandellos-2_PWR, so origen equivalent is W(17x17)
* Enrichment: 4.5
* Burnup: 43520.0 MW*d/tHM
* Cooling time: 1 min --> 12 yrs
* U weight: 72.45 MT
* Thermal power: 2940 MW for cycles 7-10, and 3070 MW for cycle 11
* Avg power for ORIGEN is in MW/MTHM, so 40.58 then 42.37 -- did avg of 41 for sim

In [5]:
XY[0:6]

,cs137/cs133,cs134/cs137,cs135/cs137,ba136/ba138,sm150/sm149,sm152/sm149,eu154/eu153,pu240/pu239,pu241/pu239,pu242/pu239,ReactorType,CoolingTime,Enrichment,Burnup,OrigenReactor
0,1.104905,0.114118,0.326079,0.014627,131.863068,41.178407,0.238885,0.287179,0.335385,0.128256,pwr,0.0,4.5,43520.0,w17x17
1,0.966216,0.020601,0.370350,0.015046,91.086858,28.444748,0.152608,0.285016,0.253497,0.126462,pwr,2000.0,4.5,43520.0,w17x17
2,0.935811,0.013437,0.382383,0.015046,91.086858,28.444748,0.136641,0.285570,0.237357,0.126483,pwr,2500.0,4.5,43520.0,w17x17
3,0.906757,0.008756,0.394635,0.015046,91.086858,28.444748,0.122392,0.285907,0.222016,0.126483,pwr,3000.0,4.5,43520.0,w17x17
4,0.879054,0.005706,0.407071,0.015046,91.086858,28.444748,0.109606,0.286244,0.207856,0.126483,pwr,3500.0,4.5,43520.0,w17x17
5,0.851351,0.003721,0.420317,0.015046,91.086858,28.444748,0.098155,0.286581,0.194370,0.126483,pwr,4000.0,4.5,43520.0,w17x17


## First, try to find best prediction via MLL

In [6]:
pred_df = mll_testset(XY[0:6].copy(), test, True, unc, lbls)

In [7]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
pred_df

,sim_idx,ReactorType,CoolingTime,Enrichment,Burnup,OrigenReactor,pred_idx,pred_ReactorType,pred_CoolingTime,pred_Enrichment,pred_Burnup,pred_OrigenReactor,MaxLogLL,MaxLLUnc,2ndMaxLogLL,2ndMaxLLUnc,CDF_LogLL_0.9998,CDF_LLUnc_0.9998,CDF_LogLL_0.9988,CDF_LLUnc_0.9988,CDF_LogLL_0.95,CDF_LLUnc_0.95,CDF_LogLL_0.9,CDF_LLUnc_0.9,CDF_LogLL_0.5,CDF_LLUnc_0.5,CDF_LogLL_0.1,CDF_LLUnc_0.1,CDF_LogLL_0.01,CDF_LLUnc_0.01
0,VAN-2|EF05|WZR0058|E58-88,pwr,0.0,4.5,43520.0,Vandellos-2_PWR,2,pwr,2500.0,4.5,43520.0,w17x17,-90.270654,24.088133,-121.709173,28.160071,-90.302092,178.254066,-90.459285,177.690869,-98.130283,150.206835,-105.989913,122.046964,-257.155033,33.48377,-837.155248,25.737647,-1241.349741,24.253084


## Second, dig in and look at ratio diffs

In [11]:
def logpdf_calc(sim_row, test_sample, unc):
    y_sim = sim_row[test_sample>0].values.tolist()
    std = sim_row.multiply(unc)[test_sample>0].values.tolist()
    y_mes = test_sample[test_sample>0].values.tolist()
    logpdf_list = stats.norm.logpdf(y_sim, loc=y_mes, scale=std)
    return logpdf_list

In [12]:
test_sample = test.drop(columns=lbls).squeeze()
test_answer = test[lbls].squeeze()
sim_row = XY.loc[2].drop(lbls).squeeze()
logpdf = logpdf_calc(sim_row, test_sample, unc)
logpdf = pd.Series(logpdf, index=sim_row[test_sample>0].index, name='LogPDF')

In [13]:
df = pd.DataFrame([test_sample, sim_row])
df.loc['Abs Diff'] = np.abs(df.loc[2] - df.loc['VAN-2|EF05|WZR0058|E58-88'])
df.loc['% Diff'] = df.loc['Abs Diff'] * 100 / df.loc['VAN-2|EF05|WZR0058|E58-88']

In [14]:
df.loc['LogPDF'] = logpdf

In [15]:
#5 honed in cools
df

,cs137/cs133,cs134/cs137,cs135/cs137,ba136/ba138,sm150/sm149,sm152/sm149,eu154/eu153,pu240/pu239,pu241/pu239,pu242/pu239
VAN-2|EF05|WZR0058|E58-88,1.021429,0.012867,0.548252,0.000000,97.640118,36.873156,0.124812,0.390879,0.184039,0.090554
2,0.935811,0.013437,0.382383,0.015046,91.086858,28.444748,0.136641,0.285570,0.237357,0.126483
Abs Diff,0.085618,0.000570,0.165869,0.015046,6.553260,8.428409,0.011829,0.105310,0.053318,0.035930
% Diff,8.382158,4.427484,30.254181,inf,6.711647,22.857844,9.477605,26.941728,28.970814,39.677799
LogPDF,0.469035,6.027037,-34.594492,NaN,-3.470242,-18.830866,2.568277,-23.868260,-6.576812,-11.994331


In [13]:
#5 more cools
df

,cs137/cs133,cs134/cs137,cs135/cs137,ba136/ba138,sm150/sm149,sm152/sm149,eu154/eu153,pu240/pu239,pu241/pu239,pu242/pu239
VAN-2|EF05|WZR0058|E58-88,1.021429,0.012867,0.548252,0.000000,97.640118,36.873156,0.124812,0.390879,0.184039,0.090554
5,0.966216,0.020601,0.370350,0.015046,91.086858,28.444748,0.152608,0.285016,0.253497,0.126462
Abs Diff,0.055212,0.007734,0.177902,0.015046,6.553260,8.428409,0.027796,0.105863,0.069458,0.035908
% Diff,5.405405,60.108696,32.448980,inf,6.711647,22.857844,22.270379,27.083404,37.741058,39.654257
LogPDF,1.458100,-22.229492,-43.079532,NaN,-3.470242,-18.830866,-2.678352,-24.260029,-11.566011,-11.980451


In [39]:
#3
df

,cs137/cs133,cs134/cs137,cs135/cs137,ba136/ba138,sm150/sm149,sm152/sm149,eu154/eu153,pu240/pu239,pu241/pu239,pu242/pu239
VAN-2|EF05|WZR0058|E58-88,1.021429,0.012867,0.548252,0.000000,97.640118,36.873156,0.124812,0.390879,0.184039,0.090554
2,1.089189,0.104777,0.328536,0.015046,91.086858,28.444748,0.232061,0.283331,0.326310,0.126462
Abs Diff,0.067761,0.091910,0.219716,0.015046,6.553260,8.428409,0.107249,0.107549,0.142271,0.035908
% Diff,6.633907,714.296985,40.075708,inf,6.711647,22.857844,85.928447,27.514608,77.304979,39.654257
LogPDF,1.217294,-149.561384,-86.261379,NaN,-3.470242,-18.830866,-39.180582,-25.479522,-34.822508,-11.980451


In [17]:
#0
# do not rerun: old results for comparison
df

,cs137/cs133,cs134/cs137,cs135/cs137,ba136/ba138,sm150/sm149,sm152/sm149,eu154/eu153,pu240/pu239,pu241/pu239,pu242/pu239
VAN-2|EF05|WZR0058|E58-88,1.021429,0.012867,0.548252,0.000000,97.640118,36.873156,0.124812,0.390879,0.184039,9.055375e-02
0,0.940490,0.011576,0.889578,0.003664,11.772640,5.406551,0.056029,0.063934,0.010777,2.667901e-04
Abs Diff,0.080938,0.001291,0.341326,0.003664,85.867478,31.466605,0.068783,0.326945,0.173262,9.028696e-02
% Diff,7.924048,10.036816,62.257242,inf,87.942825,85.337434,55.109109,83.643443,94.143980,9.970538e+01
LogPDF,0.656886,4.046256,-27.250521,NaN,-10640.341969,-6774.296690,-296.453170,-5225.273294,-51683.799005,-2.290554e+07
